<a href="https://colab.research.google.com/github/pri1712/comment_toxicity_checker/blob/main/comment_toxicity(app_running).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing necessary Libraries

In [63]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
from google.colab import drive
import seaborn as sns
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Bidirectional,Dropout
from tensorflow.keras.metrics import Precision,Recall,CategoricalAccuracy

In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
url='https://raw.githubusercontent.com/nicknochnack/CommentToxicity/main/jigsaw-toxic-comment-classification-challenge/train.csv/train.csv'

In [4]:
df=pd.read_csv(url)

In [5]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:
df.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0
159570,fff46fc426af1f9a,"""\nAnd ... I really don't think you understand...",0,0,0,0,0,0


PreProcessing Data

In [18]:
x=df['comment_text']
print(x)

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
159566    ":::::And for the second time of asking, when ...
159567    You should be ashamed of yourself \n\nThat is ...
159568    Spitzer \n\nUmm, theres no actual article for ...
159569    And it looks like it was actually you who put ...
159570    "\nAnd ... I really don't think you understand...
Name: comment_text, Length: 159571, dtype: object


storing the comment type in y as a an array of shape (6,)

In [20]:
y=df[df.columns[2:]].values
print(y.shape)
print(type(y))

(159571, 6)
<class 'numpy.ndarray'>


In [21]:
MAXF=200000 #number of words that we will be able to store

In [54]:
vectorizer=TextVectorization(max_tokens=MAXF,output_sequence_length=1800,output_mode='int')

In [55]:
vectorizer.adapt(x.values)

In [56]:
vectorized_text=vectorizer(x.values)

In [57]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]])>

In [26]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset=dataset.cache()
dataset=dataset.shuffle(160000)
dataset=dataset.batch(16)
dataset=dataset.prefetch(8)

In [28]:
#creating training ,testing and validation data split
train=dataset.take(int(len(dataset)*0.65))
dev=dataset.skip(int(len(dataset)*0.65)).take(int(len(dataset)*0.25))
test=dataset.skip(int(len(dataset)*0.9)).take(int(len(dataset)*0.1))

In [31]:
model=Sequential([
    Embedding(MAXF+1,32),
    Bidirectional(LSTM(units=32,activation='tanh')),
    Dense(units=128,activation='relu'),
    Dense(units=256,activation='relu'),
    Dense(units=128,activation='relu'),
    Dense(units=6,activation='sigmoid') #output layer and using sigmoid to classify into 0 or 1
])

In [32]:
model.compile(loss='BinaryCrossentropy',optimizer='Adam')

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [35]:
learn=model.fit(train,epochs=5,validation_data=dev)

Epoch 1/5
6483/6483 [==============================] - 789s 120ms/step - loss: 0.0631 - val_loss: 0.0450
Epoch 2/5
6483/6483 [==============================] - 653s 101ms/step - loss: 0.0459 - val_loss: 0.0417
Epoch 3/5
6483/6483 [==============================] - 647s 100ms/step - loss: 0.0413 - val_loss: 0.0358
Epoch 4/5
6483/6483 [==============================] - 643s 99ms/step - loss: 0.0367 - val_loss: 0.0348
Epoch 5/5
6483/6483 [==============================] - 655s 101ms/step - loss: 0.0342 - val_loss: 0.0305


In [37]:
learn.history

{'loss': [0.06309039145708084,
  0.04593508690595627,
  0.0412570983171463,
  0.036659304052591324,
  0.034221064299345016],
 'val_loss': [0.04496872425079346,
  0.04174778610467911,
  0.035782478749752045,
  0.03482150658965111,
  0.030519291758537292]}

In [38]:
batchx,batchy=test.as_numpy_iterator().next()
print((model.predict(batchx)>0.5).astype(int))
print(batchy)

1/1 [==============================] - 1s 852ms/step
[[0 0 0 0 0 0]
 [1 0 1 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
[[0 0 0 0 0 0]
 [1 0 1 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]


In [ ]:
#measuring accuracy and evaluating model

In [58]:
pre=Precision()

In [40]:
for i in test.as_numpy_iterator():
  xnew,ynew=i
  yhat=model.predict(xnew)
  yhat=yhat.flatten()
  ynew=ynew.flatten()

  pre.update_state(ynew,yhat)


1/1 [==============================] - 0s 54ms/step


In [42]:
print(f'Precision: {pre.result().numpy()}')


Precision: 0.8784908056259155


In [43]:
!pip install gradio jinja2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
 

In [44]:
import gradio as gr

In [ ]:
model.save('toxicity.h5')

In [ ]:
model=tf.keras.models.load_model('toxicity.h5')

In [45]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)

    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)

    return text


In [68]:
interface = gr.Interface(fn=score_comment,
                         inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(


In [69]:
interface.launch(share=True,debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://e0990cc65a142003e3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://e0990cc65a142003e3.gradio.live
